# Exercise Sheet 2

### 1 - Trotter-Suzuki Simulations

Consider the following Hamiltonian defined on four qubits.

$$
\tilde H = - J_x (X_0 X_1 + X_2 X_3) - J_z (Z_0 Z_2 + Z_1 Z_3) - h\sum_j X_j + Z_j
$$

Here $X_1 = I \otimes \sigma^x \otimes I \otimes I$, etc. The Hamiltonian is denoted $\tilde H$ here to distingush it from the Hadamard, $H$.

The time evolution of this Hamiltonian can be approximated by

$$
e^{i \tilde H t} \approx \left( e^{-i X_0 X_1 \, J_x t/n} e^{-i X_2 X_3 \, J_x t/n} e^{-i Z_0 Z_2 \, J_z t/n} e^{-i Z_1 Z_3 J_z \, t/n}\prod_j e^{-i(X_j +Z_j)\, ht/n} \right)^n
$$

where the approximation becomes increasing accurate as $n$ is increased.

To complete the following questions, first write a Qiskit circuit to implement such a simulation of the time evolution.

(a) For $J_x=1$ and $J_z=h=0$,$e^{i\tilde H t} \sim X_1 X_2 X_3 X_4$ for a suitable value of $t$. Determine the required value of $t$, and show that your simulation has this effect by appling to an initial $|0\rangle^{\otimes 4}$ state.

(b) For $J_z=1$ and $J_x=h=0$, $e^{i\tilde H t} \sim Z_1 Z_2 Z_3 Z_4$ for a suitable value of $t$. Determine the required value of $t$, and show that your simulation has this effect by appling to an initial $|+\rangle^{\otimes 4}$ state.

(c) For $J_x=J_z=0$ and $h=1$, $e^{i\tilde H t} \sim H_1 H_2 H_3 H_4$ for a suitable value of $t$. Determine the required value of $t$, and show that your simulation has this effect by appling to an initial $|0\rangle^{\otimes 4}$ state.

(d) Find $W$, a four-qubit Pauli observable (non-trivial on all qubits) that commutes with all the two-qubit terms in the Hamiltonian.

(e) For what value of $n$ (approximately) does the error in the simulation for the next question become on the same order as sampling error for $10^3$ shots? Assume that $n=1000$ yields a perfect approximation and use $J_x=J_z=1$, $h=2$.

(f) For $J_x=J_z=1$ and starting in an eigenstate of $W$, calculate $\langle W \rangle$ for various times in the interval $0 < t \leq 2 \pi$, and for a sufficiently large $n$. Plot the results for $h=0.1$, $h=1$ and $h=2$.




# 2 - Inverse QFT
Using Qiskit, implement an inverse Quantum Fourier transform for any general number of qubits.
* Unlike the QFT in the lecture notes, your inverse QFT should not reverse qubit
order.
* You must implement the $R_n$ rotations using only single qubit gates and
controlled-NOT gates.

To demonstrate that your circuit does what it is supposed to, you can use the following function. This prepares the state that is QFT of the binary representation of a given integer $j$ on a
given quantum register `qr``. Applying your inverse QFT, you should be able to rotate
back to the binary representation of $j$.


In [ ]:
import numpy as np

# this function creates the state prep circuit
def state_prep(j,qr):
    n = qr.size
    N = 2**n
    state = [0]*N
    for k in range(N):
        state[k] = np.exp( 1j * 2*np.pi*j*k/N ) / np.sqrt(N)
    qc_prep = QuantumCircuit(qr)
    qc_prep.initialize(state,qr)
    return qc_prep


Here's an example of using this function to show that an inverse QFT circuit works. Here I use the inverse QFT from Qiskit (which of course you cannot submit as your solution).

In [ ]:
from qiskit import transpile
from qiskit.circuit.library import QFT
from qiskit_aer import AerSimulator

# set an integer
j = 8
# and a number of qubits
n = 5

# create an inverse qft circuit
qc_qft = QFT(n, inverse=True)
# prepare the QFT of the integer on the same quanntum register
qc_prep = state_prep(j, qc_qft.qregs[0])
# create the combined circuit: first the prep, then the inverse qft
qc = qc_prep.compose(qc_qft)
# measure all qubits
qc.measure_all()

# run it, and see what integer comes out for 10 samples
# (hopefully, it's the one you put in)
backend = AerSimulator()
for string in backend.run(transpile(qc,backend), shots=10, memory=True).result().get_memory():
    print(int(string, 2))